In [1]:
import pandas as pd 
import numpy as np 

In [6]:
order_products_train = pd.read_csv('../../instacart dataset/order_products__train.csv')
order_products_prior = pd.read_csv('../../instacart dataset/order_products__prior.csv')
departments = pd.read_csv('../../instacart dataset/departments.csv')
sample_submission = pd.read_csv('../../instacart dataset/sample_submission.csv')
orders = pd.read_csv('../../instacart dataset/orders.csv')
products = pd.read_csv('../../instacart dataset/products_.csv')
aisles = pd.read_csv('../../instacart dataset/aisles.csv')


In [11]:
print(order_products_train.head(1))
print(order_products_prior.head(1))
print(departments.head(1))
print(orders.head(1))
print(products.head(1))
print(aisles.head(1))

   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
   department_id department
0              1     frozen
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   

   days_since_prior_order  
0                     NaN  
   product_id                product_name  aisle_id  department_id
0           1  Chocolate Sandwich Cookies        61             19
   aisle_id                  aisle
0         1  prepared soups salads


In [14]:
print("Looking into the dataframe aisles:")
print(aisles.isnull().sum())
print("Looking into the dataframe order_products_train:")
print(order_products_train.isnull().sum())
print("Looking into the dataframe order_products_prior:")
print(order_products_prior.isnull().sum())
print("Looking into the dataframe departments:")
print(departments.isnull().sum())

Looking into the dataframe aisles:
aisle_id    0
aisle       0
dtype: int64
Looking into the dataframe order_products_train:
order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64
Looking into the dataframe order_products_prior:
order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64
Looking into the dataframe departments:
department_id    0
department       0
dtype: int64


In [15]:
print("Looking into the dataframe orders:")
print(orders.isnull().sum())
print("Looking into the dataframe products:")
print(products.isnull().sum())

Looking into the dataframe orders:
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64
Looking into the dataframe products:
product_id       0
product_name     0
aisle_id         0
department_id    0
dtype: int64


In [16]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [65]:
transac = order_products_train.merge(products, on="product_id", how="left")
transac = transac.groupby('order_id')["product_name"].unique()
transac 

order_id
1          [Bulgarian Yogurt, Organic 4% Milk Fat Whole M...
36         [Grated Pecorino Romano Cheese, Spring Water, ...
38         [Shelled Pistachios, Organic Biologique Limes,...
96         [Roasted Turkey, Organic Cucumber, Organic Gra...
98         [Natural Spring Water, Organic Orange Juice Wi...
                                 ...                        
3421049    [Gluten Free Rice Bread, Organic Whole Grain W...
3421056    [Total Plain Greek Strained Yogurt, Homestyle ...
3421058    [Wine Infused Salame Cheese and Crackers Small...
3421063    [Organic Half & Half, No Salt Added Gluten-Fre...
3421070    [Organic Unsweetened Almond Milk, Creamy Peanu...
Name: product_name, Length: 131209, dtype: object

In [78]:
from apyori import apriori 

In [143]:
# apriori rules 
rules = apriori(transactions = transac, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2, max_length = 6)
rules = list(rules)

In [144]:
def inspect(results):
    '''
    function to put the result in well organised pandas dataframe
    '''
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

resultsinDataFrame = pd.DataFrame(inspect(rules), columns = ['Item #1', 'Item #2', 'Support', 'Confidence', 'Lift'])
resultsinDataFrame.head()

,Item #1,Item #2,Support,Confidence,Lift
0,Organic Navel Orange,Bag of Organic Bananas,0.005526,0.366162,3.103598
1,Bunched Cilantro,Large Lemon,0.003132,0.214063,3.452603
2,Bunched Cilantro,Limes,0.004253,0.290625,6.320672
3,Garlic,Large Lemon,0.003376,0.230729,3.721419
4,Jalapeno Peppers,Limes,0.004169,0.288046,6.264590


In [147]:
def recommend_next_products(current_product, num_recommendations=3):
    '''
    Recommends next products based on the current product
    '''
    recommendations = resultsinDataFrame[resultsinDataFrame['Item #1'] == current_product]
    recommendations = recommendations.sort_values(by='Confidence', ascending=False)
    recommendations = recommendations.head(num_recommendations)
    # print(f'Recommendations are {resultsinDataFrame}')
    return recommendations[['Item #2', 'Confidence', 'Lift']]

# choosing any random product 
current_product = 'Banana'
# current_product = 'Large Lemon'
num_recommendations = 3
next_products = recommend_next_products(current_product, num_recommendations)
print("Top", num_recommendations, "recommendations based on purchase of", current_product, ":")
print(next_products)

Top 3 recommendations based on purchase of Banana :
            Item #2  Confidence      Lift
29  Organic Avocado    0.242000  4.285677
28  Organic Avocado    0.221038  3.914452
27            Limes    0.209453  4.555303
